# 07. Portfolio Pre-processing

In [1]:
# Imports iniciais
import xml.etree.ElementTree as ET
import numpy as np
import pandas as pd

# Pacote para ignorar mensagem de warning
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Load the XML file
tree = ET.parse('C:/Users/mauri/OneDrive/Repositorios/AI-Cop_SLR/data/raw/portfolio-135.xml')
root = tree.getroot()

# Define a empty list to store the data
port_135_list = []

# Iterate over the <record> elements and extract the information
for record in root.findall('records/record'):
    
    # Extract data from each field
    database = record.find('database').text if record.find('database') is not None else None
    ref_type = record.find('ref-type').get('name') if record.find('ref-type') is not None else None
    authors = ", ".join([author.text for author in record.findall('contributors/authors/author')]) if record.findall('contributors/authors/author') is not None else None
    title = record.find('titles/title').text if record.find('titles/title') is not None else None
    secondary_title = record.find('titles/secondary-title').text if record.find('titles/secondary-title') is not None else None
    periodical = record.find('periodical/full-title').text if record.find('periodical/full-title') is not None else None
    pages = record.find('pages').text if record.find('pages') is not None else None
    volume = record.find('volume').text if record.find('volume') is not None else None
    issue = record.find('issue').text if record.find('issue') is not None else None
    keywords = ", ".join([keyword.text for keyword in record.findall('keywords/keyword')]) if record.findall('keywords/keyword') is not None else None
    year = record.find('dates/year').text if record.find('dates/year') is not None else None
    DOI = record.find('electronic-resource-num').text if record.find('electronic-resource-num') is not None else None
    pdf_url = record.find('urls/pdf-urls/url').text if record.find('urls/pdf-urls/url') is not None else None
    web_url = record.find('urls/web-urls/url').text if record.find('urls/web-urls/url') is not None else None
    abstract = record.find('abstract').text if record.find('abstract') is not None else None
    
    # Add the extracted data to the list
    port_135_list.append([database, ref_type, authors, title, secondary_title, periodical, pages, volume, issue, keywords, year, DOI, pdf_url, web_url, abstract])

# Create a DataFrame with the data
df = pd.DataFrame(port_135_list, columns=['database', 'ref-type', 'authors', 'title', 'secondary-title', 'periodical', 'pages', 'volume', 'issue', 'keywords', 'year', 'DOI', 'pdf_url', 'web_url', 'abstract'])

# Check the result
df.head(3)

,database,ref-type,authors,title,secondary-title,periodical,pages,volume,issue,keywords,year,DOI,pdf_url,web_url,abstract
0,PORTFOLIO-135.enl,Journal Article,"Beccali, Marco, Ciulla, Giuseppina, Lo Brano, ...",Artificial neural network decision support too...,Energy,Energy,1201-1218,137,None,"ANN, Decision support tool, Energy audit, Ener...",2017,10.1016/j.energy.2017.05.200,internal-pdf://2017 - Beccali et al. - Energy.pdf,https://linkinghub.elsevier.com/retrieve/pii/S...,The public buildings sector represents one of ...
1,PORTFOLIO-135.enl,Journal Article,"Giest, Sarah",Big data for policymaking: fad or fasttrack?,Policy Sciences,Policy Sciences,367 - 382,50,3,"Big data, Data readiness, Digital-era governan...",2017,10.1007/s11077-017-9293-1,internal-pdf://2017 - Giest - Policy Sciences.pdf,https://www.scopus.com/inward/record.uri?eid=2...,The buzz surrounding big data has taken shape ...
2,PORTFOLIO-135.enl,Journal Article,"Shah, Naimatullah, Irani, Zahir, Sharif, Amir M.",Big data in an HR context: Exploring organizat...,Journal of Business Research,Journal of Business Research,366-378,70,None,"Big data, Employee readiness, Extrinsic and in...",2017,10.1016/j.jbusres.2016.08.010,"internal-pdf://2017 - Shah, Irani, Sharif - Jo...",https://linkinghub.elsevier.com/retrieve/pii/S...,This research highlights a contextual applicat...


In [3]:
types_list = df['ref-type'].value_counts()
print(types_list)

ref-type
Journal Article    135
Name: count, dtype: int64


In [4]:
df.value_counts(df['secondary-title'] == df['periodical'])

True    135
Name: count, dtype: int64

In [5]:
# Como as colunas 'secondary-title' e 'periodical' são idênticas, vamos excluir uma e renomear a outra.
df.drop('periodical', axis=1, inplace=True)

In [6]:
df.rename(columns={'secondary-title':'journal'}, inplace=True)

In [7]:
print(df.columns)

Index(['database', 'ref-type', 'authors', 'title', 'journal', 'pages',
       'volume', 'issue', 'keywords', 'year', 'DOI', 'pdf_url', 'web_url',
       'abstract'],
      dtype='object')


In [8]:
df.describe()

,database,ref-type,authors,title,journal,pages,volume,issue,keywords,year,DOI,pdf_url,web_url,abstract
count,135,135,135,135,135,116,126,98,135,135,135,135,134,134
unique,1,1,133,135,68,116,71,14,117,8,135,135,134,134
top,PORTFOLIO-135.enl,Journal Article,"Ingrams, Alex",Artificial neural network decision support too...,Government Information Quarterly,1201-1218,39,1,,2022,10.1016/j.energy.2017.05.200,internal-pdf://2017 - Beccali et al. - Energy.pdf,https://linkinghub.elsevier.com/retrieve/pii/S...,The public buildings sector represents one of ...
freq,135,135,2,1,23,1,10,25,18,29,1,1,1,1


In [9]:
# Converter para strings e remover os símbolos '[' e ']'
df['authors'] = df['authors'].astype(str).str.replace('[', '', regex=False).str.replace(']', '', regex=False)
df['keywords'] = df['keywords'].astype(str).str.replace('[', '', regex=False).str.replace(']', '', regex=False)

# Verificar o resultado
print(df[['authors', 'keywords']].head())

                                             authors  \
0  Beccali, Marco, Ciulla, Giuseppina, Lo Brano, ...   
1                                       Giest, Sarah   
2   Shah, Naimatullah, Irani, Zahir, Sharif, Amir M.   
3                    Desouza, Kevin C., Jacob, Benoy   
4  Klievink, Bram, Romijn, Bart-Jan, Cunningham, ...   

                                            keywords  
0  ANN, Decision support tool, Energy audit, Ener...  
1  Big data, Data readiness, Digital-era governan...  
2  Big data, Employee readiness, Extrinsic and in...  
3  big data, policy analysis, public management, ...  
4  Assessment, Big data, Bold Readiness, E-govern...  


In [10]:
def replace_alternate_commas(text):
    result = []
    comma_count = 0  # Contador para rastrear cada vírgula

    # Percorrer cada caractere da string
    for char in text:
        if char == ',':
            comma_count += 1
            # Substituir a vírgula por ponto e vírgula em contagens pares
            if comma_count % 2 == 0:
                result.append(';')
            else:
                result.append(char)
        else:
            result.append(char)

    # Retornar a string modificada
    return ''.join(result)

# Aplicar a função na coluna 'authors'
df['authors'] = df['authors'].apply(replace_alternate_commas)

# Verificar o resultado
print(df['authors'].head())

0    Beccali, Marco; Ciulla, Giuseppina; Lo Brano, ...
1                                         Giest, Sarah
2     Shah, Naimatullah; Irani, Zahir; Sharif, Amir M.
3                      Desouza, Kevin C.; Jacob, Benoy
4    Klievink, Bram; Romijn, Bart-Jan; Cunningham, ...
Name: authors, dtype: object


In [11]:
# Converter as colunas em listas
df['authors'] = df['authors'].str.split(';').apply(lambda x: [a.strip() for a in x])
df['keywords'] = df['keywords'].str.split(',').apply(lambda x: [k.strip() for k in x])

# Explodir as colunas sequencialmente
df_exploded = df.explode('authors').explode('keywords')

# Opcional: Resetar o índice
df_exploded.reset_index(drop=True, inplace=True)

# Visualizar o DataFrame final
print(df_exploded)

               database         ref-type          authors  \
0     PORTFOLIO-135.enl  Journal Article   Beccali, Marco   
1     PORTFOLIO-135.enl  Journal Article   Beccali, Marco   
2     PORTFOLIO-135.enl  Journal Article   Beccali, Marco   
3     PORTFOLIO-135.enl  Journal Article   Beccali, Marco   
4     PORTFOLIO-135.enl  Journal Article   Beccali, Marco   
...                 ...              ...              ...   
1716  PORTFOLIO-135.enl  Journal Article   Plewa, Carolin   
1717  PORTFOLIO-135.enl  Journal Article  Reaiche, Carmen   
1718  PORTFOLIO-135.enl  Journal Article  Reaiche, Carmen   
1719  PORTFOLIO-135.enl  Journal Article  Reaiche, Carmen   
1720  PORTFOLIO-135.enl  Journal Article  Reaiche, Carmen   

                                                  title  \
0     Artificial neural network decision support too...   
1     Artificial neural network decision support too...   
2     Artificial neural network decision support too...   
3     Artificial neural network

In [ ]:
df_exploded.to_excel('C:/Users/mauri/OneDrive/Repositorios/AI-Cop_SLR/data/processed/_11_portfolio-135-final.xlsx', index=False)

---